# Código para filtrar el TOTAL de fallecidos por COVID-19 de los 24 departamentos del Perú

Empezamos cargando las librerías que necesitamos y leyendo nuestro csv actualizado de fallecidos por COVID-19.

In [1]:
import pandas as pd
import numpy as np
import functions as fn


def just_cities(fal_url):
    """
    Función para filtrar a cada fallecido reportado en su año
    y semana epidemiológica en ciudad (Fallecidos = fal, Vacunados = vac)
    """     
    # Seleccionamos solo las col 'FECHA_FALLECIMIENTO' y 'DEPARTAMENTO'      
    fal_col = ['FECHA_FALLECIMIENTO', 'DEPARTAMENTO']   
    df_fal = fn.read_largeCSV_file(fal_url, ';', fal_col)
    fn.variable_fecha(df_fal, 'FECHA_FALLECIMIENTO')
    fn.date_to_epiweek(df_fal,'FECHA_FALLECIMIENTO')
    df_fal['fallecido'] = 1                                         
    df_fal['fallecido'].apply(np.int8) 
    df_fal.info()
    
    return df_fal


fal_url = "RawData/fallecidos_covid.csv"
df = just_cities(fal_url)
del fal_url

Read csv with dask:  0.6090712547302246 sec
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200246 entries, 0 to 200245
Columns: 2 entries, FECHA_FALLECIMIENTO to DEPARTAMENTO
dtypes: category(1), int64(1)
memory usage: 1.7 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200246 entries, 0 to 200245
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   DEPARTAMENTO  200246 non-null  category
 1   epi_year      200246 non-null  int64   
 2   epi_week      200246 non-null  int64   
dtypes: category(1), int64(2)
memory usage: 3.2 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200246 entries, 0 to 200245
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   DEPARTAMENTO  200246 non-null  category
 1   epi_year      200246 non-null  int64   
 2   epi_week      200246 non-null  int64   
 3   fallecido     200246 non-null  int64   
dtype

In [2]:
print(df.head())

  DEPARTAMENTO  epi_year  epi_week  fallecido
0         LIMA      2021        17          1
1         LIMA      2021        17          1
2     AYACUCHO      2021        17          1
3         LIMA      2021        16          1
4     AREQUIPA      2021        31          1


Realizamos un crosstab de del total de fallecidos por COVID-19 por departamento del Perú, por cada semana y año epidemiológico.

In [3]:
epi_ciudades = pd.crosstab(index=[df['epi_year'], df['epi_week']],
                           columns=[df['fallecido'], df['DEPARTAMENTO']],
                           margins = True)

epi_ciudades.to_csv('Data/fallecidosXciudadesXsemanasEpi.csv')
                           

Ahora realizamos un crosstab del TOTAL de fallecidos por departamento.

In [4]:
fal_ciudades = pd.crosstab(index = df['DEPARTAMENTO'],
                           columns = df['fallecido'],
                           margins = True)
del fal_ciudades["All"]
fal_ciudades.columns = ['fallecidos']
fal_ciudades.rename(index={'All': 'PERÚ'})

,fallecidos
DEPARTAMENTO,
AMAZONAS,1258
ANCASH,6674
APURIMAC,1510
AREQUIPA,9680
AYACUCHO,2121
CAJAMARCA,4135
CALLAO,9982
CUSCO,4794
HUANCAVELICA,1161


In [7]:
# https://es.wikipedia.org/wiki/Anexo:Departamentos_del_Per%C3%BA_por_poblaci%C3%B3n
dict_dep = {
    "AMAZONAS"	:	426806,
    "ANCASH"	:	1180638,
    "APURIMAC"	:	430736,
    "AREQUIPA"	:	1497438,
    "AYACUCHO"	:	668213,
    "CAJAMARCA"	:	1453711,
    "CALLAO"	:	1129854,
    "CUSCO"	    :	1357075,
    "HUANCAVELICA":	365317,
    "HUANUCO"	:	760267,
    "ICA"	    :	975182,
    "JUNIN"	    :	1361467,
    "LALIBERTAD":	2016771,
    "LAMBAYEQUE":	1310785,
    "LIMA"	    :	10628470,
    "LORETO"	:	1027559,
    "MADREDEDIOS":	173811,
    "MOQUEGUA"	:	192740,
    "PASCO"	    :	271904,
    "PIURA"	    :	2047954,
    "PUNO"	    :	1237997,
    "SANMARTIN"	:	899648,
    "TACNA"	    :	370974,
    "TUMBES"	:	251521,
    "UCAYALI"	:	589110,
    "PERÚ"  	:	32625948,
}



no_habitantes = [426806,
                 1180638,
                 430736,
                 1497438,
                 668213,
                 1453711,
                 1129854,
                 1357075,
                 365317,
                 760267,
                 975182,
                 1361467,
                 2016771,
                 1310785,
                 10628470,
                 1027559,
                 173811,
                 192740,
                 271904,
                 2047954,
                 1237997,
                 899648,
                 370974,
                 251521,
                 589110]

no_habitantes.append(sum(no_habitantes))
fal_ciudades['no_habitantes'] = no_habitantes
del no_habitantes

fal_ciudades['tasa_mortalidad'] = (fal_ciudades['fallecidos']/fal_ciudades['no_habitantes'])*100

fal_ciudades.to_csv('Data/TOTAL_fallecidosXciudades.csv')